In [14]:
import os
import numpy as np
import cv2

from matplotlib import pyplot as plt
import matplotlib 

from time import time
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

from skimage.feature import hog

### Cargado de datos

In [15]:
folder = "C:/Users/maria/Downloads/ages/train" 

nclasses = 0
nperclass = []
classlabels = []
X = []
Y = []

dpi = matplotlib.rcParams['figure.dpi']

for class_name in os.listdir(folder):
    nclasses += 1
    nsamples = 0

    class_folder = os.path.join(folder, class_name)
    for file_name in os.listdir(class_folder):
        if file_name.endswith('.jpg'):
            image = cv2.imread (os.path.join(class_folder, file_name),)
            image = cv2.resize(image, (128, 128))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            features = hog(gray, 
                           orientations=9,
                           pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2),
                           block_norm='L2-Hys')

            X.append(features)
            Y.append(nclasses-1)
  
            nsamples += 1
    nperclass.append(nsamples)
    classlabels.append(class_name)
X = np.array(X,dtype='float32')
Y = np.array(Y,dtype='float64')


n_samples , n_features = X.shape

class_names = np.array(classlabels)
n_classes = class_names.shape[0]

print("Dataset info:")
print("# features: %d" % n_features)
print("# classes: %d" % n_classes)
print("classes %s" % classlabels)
print("samples per class %s" % str(nperclass)[1:-1] )

Dataset info:
# features: 8100
# classes: 5
classes ['18-20', '21-30', '31-40', '41-50', '51-60']
samples per class 30, 30, 30, 30, 30


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

print("# samples in training set %d" % X_train.shape[0])
print("# samples in test set %d" % X_test.shape[0])

# samples in training set 105
# samples in test set 45


In [21]:
print("SVM classifier based on HOG\nTraining...")
t0 = time()

model_svm = SVC(kernel='rbf', C=5, gamma='scale')
model_svm.fit(X_train, y_train)

print("done in %0.3fs" % (time() - t0))

print("Predicting...")
t0 = time()
y_pred = model_svm.predict(X_test)
print("done in %0.3fs" % (time() - t0))

print("Classification results:")
print(classification_report(y_test, y_pred, target_names=class_names))

print("Precision:  %0.3f, Recall:  %0.3f" %
      (precision_score(y_test, y_pred, average='macro'),
       recall_score(y_test, y_pred, average='macro')))

print("Confusion matrix")
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


SVM classifier based on HOG
Training...
done in 0.079s
Predicting...
done in 0.037s
Classification results:
              precision    recall  f1-score   support

       18-20       0.67      0.15      0.25        13
       21-30       0.50      0.33      0.40         9
       31-40       0.25      0.20      0.22        10
       41-50       0.14      0.75      0.24         4
       51-60       0.14      0.11      0.12         9

    accuracy                           0.24        45
   macro avg       0.34      0.31      0.25        45
weighted avg       0.39      0.24      0.25        45

Precision:  0.340, Recall:  0.310
Confusion matrix
[[2 3 3 3 2]
 [1 3 1 3 1]
 [0 0 2 6 2]
 [0 0 0 3 1]
 [0 0 2 6 1]]


## Entrenamiento